In [2]:
import polars as pl
import re
from unidecode import unidecode
pl.Config.set_fmt_str_lengths(200)

polars.config.Config

### Preprocess data

In [3]:
def remove_non_ascii(string):
    return string.encode('ascii', 'ignore').decode('utf8').casefold()

In [4]:
auchan = (
    pl.read_parquet('data/raw_v2/auchan.parquet')    
    .with_columns(pl.col('brand').apply(lambda x: remove_non_ascii(x))
                .alias("brand")
    )   
    .with_columns(pl.col('name').apply(lambda x: remove_non_ascii(x))
                .alias("name")
    )
    .select(pl.col('brand').str.to_uppercase().alias('brand_auchan'), 
            'ean', 
            pl.col('name').str.to_uppercase().alias('name_auchan')
            )
    .unique(subset=['ean'])
)
auchan

brand_auchan,ean,name_auchan
str,str,str
"""AUCHAN""","""3596710470327""","""BTONNET GLAC VANILLE AMANDES"""
"""CAROLIN""","""3346029989412""","""SPRAY NETTOYANT MULTI-SURFACES AU SAVON NOIR 100% VGTAL"""
"""BONDUELLE""","""3083681133691""","""FARFALLES SAUMON ET FVES AU CURRY ET LA CORIANDRE"""
"""KNORR""","""8712100630040""","""SAUCE AUX CHAMPIGNONS LA CRME FRACHE EN BRIQUE"""
"""FERME D'ANCHIN""","""3490214750018""","""SOUPE AUX 10 LGUMES DU POTAGER"""
"""ELSEVE""","""3054089402597""","""TOTAL REPAIR 5 MASQUE RECONSTITUANT CHEVEUX ABIMS"""
"""SAMIA""","""3276650101535""","""HALVAS AMANDES"""
"""NESTLE""","""7613035856356""","""NATURNES BOL CAROTTES RIZ ET POULET FERMIER DS 6 MOIS"""
"""WEI MING""","""3275760000707""","""SORBET LA NOIX DE COCO"""


In [5]:
franprix = (
    pl.read_parquet('data/raw_v2/franprix.parquet')     
    .with_columns(pl.col('brand').apply(lambda x: remove_non_ascii(x))
                .alias("brand")
    )
    .with_columns(pl.col('name').apply(lambda x: remove_non_ascii(x))
                .alias("name")
    )
    .select(pl.col('brand').str.to_uppercase().alias('brand_franprix'), 
            'ean', 
            pl.col('name').str.to_uppercase().alias('name_franprix')
            )
    .unique(subset=['ean'])
)
franprix

brand_franprix,ean,name_franprix
str,str,str
"""MARLETTE""","""3760363690214""","""PRPARATION BIO POUR PANCAKES & GAUFRES"""
"""CRAL BIO""","""3175681152137""","""QUINOA NATURE BIO"""
"""FRANPRIX""","""3263858760923""","""SANDWICH ROSETTE BEURRE"""
"""BROSSARD""","""3660140846455""","""BROWNIE PPITES CHOCOLAT"""
"""ROYCO""","""3036812020681""","""SOUPE INSTANTANE CRME DE CHAMPIGONS"""
"""FRANPRIX""","""3263857728115""","""CINSAULT PAYS D'OC, VIN ROS"""
"""FRANPRIX""","""3263856489819""","""FILETS DE COLIN D'ALASKA PANS"""
"""APTONIA""","""3583787609163""","""BOISSON ISOTONIQUE ORANGE"""
"""FRANPRIX""","""3263859356217""","""COMT AOP RP"""


In [6]:
def extract_brand(product_name):
    product_name_processed = product_name.replace('"', "'").strip()
    reversed_product_name = product_name_processed[::-1]
    brand_reverse_lst = []
    ind = 0
    pattern = ".?\'(.*)\'$"
    
    # Get brand in capital letters
    for character in reversed_product_name:
        if character.isupper() or character.isdigit() or character in [" ", "'", "-", "&", "?", "!", "."]:
            brand_reverse_lst.append(character) 
            ind += 1
        else:
            break

    if len(brand_reverse_lst) > 0:
        brand = ''.join(brand_reverse_lst).strip()[::-1]

        brand = re.sub("^'", "", brand)
        brand = re.sub("'$", "", brand)
        brand = re.sub("^-", "", brand)
        brand = re.sub("^\.", "", brand)
        brand = re.sub("^\d+", "", brand)
        brand = brand.strip()
        brand = brand.rstrip()

        if re.search("^\-\w?$", brand) \
           or re.search("^X.*\d*$", brand) \
           or re.search("^\d+.*L\.*$", brand)\
           or re.search("^\d+.*U\.*$", brand)\
           or re.search("^\d+.*KG\.*$", brand)\
           or re.search("^\d+.*G\.*$", brand) \
           or brand.isdigit() \
           or re.sub("\-|\.|\ |\?", "",brand).isdigit() \
           or re.sub("\.", "",brand) == '':
            brand = None
    
    # Get brand in quotes
    elif len(brand_reverse_lst) < 2 and re.search(pattern, product_name):
        brand = re.search(pattern, product_name).group(1).strip()
        
    else :
        brand = None

    return brand

In [7]:
carrefour = (
    pl.read_parquet('data/raw_v2/carrefour.parquet')
    .with_columns(pl.col('name').apply(lambda x: extract_brand(x))
                .alias("brand")
    ) 
    .with_columns(pl.col('brand').apply(lambda x: remove_non_ascii(x))
                .alias("brand")
    )
    .with_columns(pl.col('name').apply(lambda x: remove_non_ascii(x))
                .alias("name")
    )
    .select(pl.col('brand').str.to_uppercase().alias('brand_carrefour'), 
            'ean', 
            pl.col('name')
            .str.to_uppercase().alias('name_carrefour')
            )
)
carrefour
# carrefour.write_parquet('data/raw_v2/carrefour.parquet')

brand_carrefour,ean,name_carrefour
str,str,str
"""CARREFOUR LE MARCHE""","""5400101069220""","""NOIX DE PCAN CARREFOUR LE MARCHE"""
"""LU""","""3041090063114""","""BISCOTTES AU FROMENT CRACOTTE LU"""
null,"""3701597301583""","""BROCHETTE DE GIGOT D'AGNEAU- 4 PICES DE 160/180G """
"""K'BICHOO""","""3760256900062""","""BONBONS TTINES HALAL K'BICHOO"""
"""OREO""","""7622210477439""","""BISCUITS FOURRS LA VANILLE GOLDEN OREO"""
"""COCA-COLA""","""5449000239785""","""SODA SANS SUCRES COCA-COLA"""
"""DD""","""3504900022034""","""JAMBON D'AUCH SUPRIEUR DD"""
null,"""3770006004018""","""NOUGAT TRADITIONNEL"""
"""FQC FILIERE QUALITE CARREFOUR""","""3245416059403""","""CTE DE PORC CHINE GRILLER FQC FILIERE QUALITE CARREFOUR"""


In [46]:
dataset = (
    auchan
    .join(franprix, on = 'ean', how='outer')
    .join(carrefour, on = 'ean', how='outer')
    .select(['ean', 'name_auchan', 'name_franprix', 'name_carrefour', 'brand_auchan', 'brand_franprix',  'brand_carrefour'])
)
dataset
dataset.write_csv('data/raw_v2/brands.csv', separator=';')

In [47]:
training_0 = (
    dataset
    .filter(pl.col('brand_auchan').is_not_null())
    .filter(pl.col('brand_franprix').is_not_null())
    .filter(pl.col('brand_franprix') != pl.col('brand_auchan'))
    .select(pl.col('brand_franprix').alias('brand_left'), pl.col('brand_auchan').alias('brand_right'))
    .unique()
)

training_1 = (
    dataset
    .filter(pl.col('brand_auchan').is_not_null())
    .filter(pl.col('brand_carrefour').is_not_null())
    .filter(pl.col('brand_carrefour') != pl.col('brand_auchan'))
    .select(pl.col('brand_carrefour').alias('brand_left'), pl.col('brand_auchan').alias('brand_right'))
    .unique()
)

training_2 = (
    dataset
    .filter(pl.col('brand_carrefour').is_not_null())
    .filter(pl.col('brand_franprix').is_not_null())
    .filter(pl.col('brand_franprix') != pl.col('brand_carrefour'))
    .select(pl.col('brand_franprix').alias('brand_left'), pl.col('brand_carrefour').alias('brand_right'))
    .unique()
)

training = pl.concat([training_0, training_1, training_2]).unique()
display(training)
training.write_csv('data/raw_v2/training_2.csv', separator=';')

brand_left,brand_right
str,str
"""CONNTABLE""","""CONNETABLE"""
"""HARIBO""","""MIKO"""
"""SAINT MAMET""","""ST MAMET"""
"""SAVANE""","""BROSSARD"""
"""PELICAN BLONDE""","""PELICAN"""
"""APRIVRAIS""","""APERIVRAIS"""
"""MEN EXPERT""","""L'OREAL"""
"""MISSION SAINT VINCENT""","""PIERRE CHANAU"""
"""CHARLES ET ALICE""","""CHARLES & ALICE"""


In [48]:
dataset.describe().filter(pl.col('describe').is_in(['count', 'null_count']))

describe,ean,name_auchan,name_franprix,name_carrefour,brand_auchan,brand_franprix,brand_carrefour
str,str,str,str,str,str,str,str
"""count""","""92221""","""92221""","""92221""","""92221""","""92221""","""92221""","""92221"""
"""null_count""","""0""","""47095""","""77924""","""41745""","""47095""","""78849""","""60699"""


In [10]:
df_cross_join_0 = (
    auchan
    .select(pl.col('brand_auchan').alias('brand_left'))
    .unique()
    .join(franprix
          .select(pl.col('brand_franprix').alias('brand_left'))
          .unique(), 
          how="cross")
)

df_cross_join_1 = (
    auchan
    .select(pl.col('brand_auchan').alias('brand_left'))
    .unique()
    .join(carrefour
          .select(pl.col('brand_carrefour').alias('brand_left'))
          .unique(), 
          how="cross")
)

df_cross_join_2 = (
    carrefour
    .select(pl.col('brand_carrefour').alias('brand_left'))
    .unique()
    .join(franprix
          .select(pl.col('brand_franprix').alias('brand_left'))
          .unique(), 
          how="cross")
)

cross_join = pl.concat([df_cross_join_0, df_cross_join_1, df_cross_join_2]).unique().sample(200)
display(cross_join)
cross_join.write_csv('data/raw_v2/cross_join.csv', separator=';')

brand_left,brand_left_right
str,str
"""BRASSERIE DU COMT""","""AOC MENETOU SALON"""
"""AURLANE""","""TAUREAU AILE"""
"""LE JARDIN D'ORANTE""","""YUMI"""
"""GLORIA""","""8.6"""
"""EQUIPE DE FRANCE JO 2024""","""SIGNAL"""
"""LES MOULINS DE PROVENCE""","""DOMAINE DU ROUCAS DE SAINT PIERRE"""
"""BRETS""","""AOP CLOCHE D'OR"""
"""LA COLOMBIERE""","""MILLER"""
"""FOREST STYLE""","""SAINT MARC"""
